# Experiment 2 – Key Management Service + RBAC

**Goal:** Test Role-Based Access Control (RBAC) through having an external Key Management Service (KMS) manage keys for each role. Use the keys to decrypt the data if possible.

In [1]:
# Imports
from dotenv import load_dotenv
from constants import *
from utility_functions import *
from constants import *

# Load environment variables
load_dotenv()

True

In [2]:
# Setup for connection towards AWS KMS
encryption_config = pe.EncryptionConfiguration(
    footer_key="file-access-key",             
    column_keys={
        "salary-key": ["Salary"],     
        "password-key": ["Password"] 
    },
    plaintext_footer=False         
)

In [3]:
# Get credentials for roles
engineer_credentials = get_aws_credentials(ROLE_ENGINEER)
hr_credentials = get_aws_credentials(ROLE_HR)
admin_credentials = get_aws_credentials(ROLE_ADMIN)

# Get aws client for S3 and KMS
engineer_kms_client = create_aws_client_for(engineer_credentials, "kms")
hr_kms_client = create_aws_client_for(hr_credentials, "kms")
admin_kms_client = create_aws_client_for(admin_credentials, "kms")

engineer_s3_client = create_aws_client_for(engineer_credentials, "s3")
hr_s3_client = create_aws_client_for(hr_credentials, "s3")
admin_s3_client = create_aws_client_for(admin_credentials, "s3")

Assumed ENGINEER role successfully.
Assumed HR role successfully.
Assumed ADMIN role successfully.


In [4]:
file = get_data(engineer_s3_client, EMPLOYEE_DATA_ENCRYPTED_KEY)
df = decrypt_parquet(file, engineer_kms_client)
df.head(3)


 successfully loaded 'employee_data_encrypted.parquet' from S3 bucket 's3-rbac-in-data-lakes-experiments'
File columns: ['ID', 'Name', 'Email', 'Department', 'Salary', 'Password']
Skipping column Salary due to decryption error: An error occurred (AccessDeniedException) when calling the Decrypt operation: User: arn:aws:sts::501994300007:assumed-role/ENGINEER/ENGINEER-notebook-session is not authorized to perform: kms:Decrypt on resource: arn:aws:kms:eu-north-1:501994300007:key/86c41c3f-fc21-4730-a20b-b755e5b63ebb because no identity-based policy allows the kms:Decrypt action
Skipping column Password due to decryption error: An error occurred (AccessDeniedException) when calling the Decrypt operation: User: arn:aws:sts::501994300007:assumed-role/ENGINEER/ENGINEER-notebook-session is not authorized to perform: kms:Decrypt on resource: arn:aws:kms:eu-north-1:501994300007:key/5722129f-2136-4ef4-8b53-5a242b553f34 because no identity-based policy allows the kms:Decrypt action


,ID,Name,Email,Department
0,1,Alice,alice@example.com,HR
1,2,Bob,bob@example.com,Engineering
2,3,Charlie,charlie@example.com,Marketing


In [5]:
file = get_data(hr_s3_client, EMPLOYEE_DATA_ENCRYPTED_KEY)
df = decrypt_parquet(file, hr_kms_client)
df.head(3)


 successfully loaded 'employee_data_encrypted.parquet' from S3 bucket 's3-rbac-in-data-lakes-experiments'
File columns: ['ID', 'Name', 'Email', 'Department', 'Salary', 'Password']
Skipping column Password due to decryption error: An error occurred (AccessDeniedException) when calling the Decrypt operation: User: arn:aws:sts::501994300007:assumed-role/HR/HR-notebook-session is not authorized to perform: kms:Decrypt on resource: arn:aws:kms:eu-north-1:501994300007:key/5722129f-2136-4ef4-8b53-5a242b553f34 because no identity-based policy allows the kms:Decrypt action


,ID,Name,Email,Department,Salary
0,1,Alice,alice@example.com,HR,55000
1,2,Bob,bob@example.com,Engineering,72000
2,3,Charlie,charlie@example.com,Marketing,63000


In [8]:
file = get_data(admin_s3_client, EMPLOYEE_DATA_ENCRYPTED_KEY)
df = decrypt_parquet(file, admin_kms_client)
df.head(3)


 successfully loaded 'employee_data_encrypted.parquet' from S3 bucket 's3-rbac-in-data-lakes-experiments'
File columns: ['ID', 'Name', 'Email', 'Department', 'Salary', 'Password']


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,DummyPassword1
1,2,Bob,bob@example.com,Engineering,72000,DummyPassword2
2,3,Charlie,charlie@example.com,Marketing,63000,DummyPassword3
